In [1]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import os
# import sys
# sys.path.append(os.getcwd())
from workers import *

In [4]:
data_path = '..\\Data\\In\\'
submissions_path =  '..\\Submissions\\'

In [25]:
#df.to_pickle("df.pkl")

In [5]:
df = pd.read_pickle("df.pkl")

In [6]:
df.head()

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,key_skills,employer,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags
0,2700001,Прораб (Великий Устюг),<p><strong>Обязанности:</strong></p> <p>—взаим...,2,2019-06-14,fullDay,between3And6,NaN,NaN,NaN,[],72dc8d78bbec,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, /p, p, /p, p, /p, p, /p, ..."
1,2700002,Лаборант электротехнической лаборатории,<p><strong>В связи с расширением Компании нам ...,1,2019-08-30,fullDay,between1And3,35000.0,45000.0,RUR,"[MS Office, Пользователь ПК, Деловое общение, ...",ba6dcbd0bcd1,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, br /, strong, br /, /stro..."
2,2700003,Продавец-консультант ТЦ Радуга Парк,<strong>Обязанности:</strong> <ul> <li>консуль...,3,2019-06-17,fullDay,between1And3,25000.0,32000.0,RUR,"[Грамотная речь, Коммуникабельность, Активные ...",ddc67b017661,None,True,10,full,NaN,None,NaN,"[strong, /strong, ul, li, /li, li, /li, li, /l..."
3,2700004,Тестировщик в команду Saving products,<p>Мы - команда <strong>Saving products.</stro...,1,2019-09-25,fullDay,between1And3,NaN,NaN,NaN,[],a621c7f876ec,None,True,10,full,NaN,None,NaN,"[p, strong, /strong, br /, br /, br /, br /, e..."
4,2700005,Разработчик Python/django,<p>В небольшую дружную команду требуется опытн...,1,2019-09-03,fullDay,between3And6,70000.0,80000.0,RUR,[],845fe6266ab1,"[137, 221, 296]",False,10,full,3.0,[1],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, ul,..."


In [1]:
#df['html_tags'] = pqdm(df.description, extract_html, n_jobs=10)

In [7]:
df_train = df[df.is_test == False].copy()
del df

In [8]:
#df_train.to_parquet("df_tarin.pq")

In [ ]:
# if __name__ ==  '__main__': 
#     df['html_tags'] = parallelize_on_rows(df.description, extract_html) 

In [5]:
df_train= pd.read_parquet("df_tarin.pq")

In [8]:
df_train.head(5)

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,...,employer,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags,clear_text
4,2700005,Разработчик Python/django,<p>В небольшую дружную команду требуется опытн...,1,2019-09-03,fullDay,between3And6,70000.0,80000.0,RUR,...,845fe6266ab1,"[137, 221, 296]",False,10,full,3.0,[1],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, ul,...",В небольшую дружную команду требуется опытный ...
5,2700006,Торговый представитель,<p>Обязанности: В компанию дистрибьютор продук...,53,2019-10-01,fullDay,noExperience,30000.0,60000.0,RUR,...,23ead4a8f164,"[149, 242]",False,10,full,2.0,[17],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, p, ...",Обязанности: В компанию дистрибьютор продуктов...
6,2700007,Менеджер по оптовым продажам тканей,<p>СТАРТЕКС - крупная текстильная компания. Ли...,1,2019-03-13,fullDay,between1And3,50000.0,NaN,RUR,...,9e3008e9f553,"[149, 196, 242, 324, 538]",False,10,full,5.0,[17],1.0,"[p, /p, p, /p, p, /p, p, strong, /strong, /p, ...",СТАРТЕКС - крупная текстильная компания. Лидер...
8,2700010,Врач клинической лабораторной диагностики (Бак...,<p><em><strong>Компания KDL Клинико-Диагностич...,1,2019-01-28,shift,between1And3,58000.0,70000.0,RUR,...,49f3280f53b6,"[398, 537, 587]",False,10,full,3.0,[13],1.0,"[p, em, strong, /strong, /em, em, strong, /str...",Компания KDL Клинико-Диагностические лаборатор...
12,2700015,Швея-портной,<p>Требуется швея-лаборант с навыками закройщи...,1,2019-03-22,fullDay,between3And6,40000.0,50000.0,RUR,...,24e6806caa88,"[129, 157, 516]",False,10,full,3.0,"[18, 11, 29]",3.0,"[p, /p, p, strong, /strong, /p, ul, li, /li, l...",Требуется швея-лаборант с навыками закройщика ...


In [7]:
if __name__ ==  '__main__':
    df_train['clear_text'] = parallelize_on_rows(df_train.description, remove_html) 

In [ ]:
if __name__ ==  '__main__': 
    df_train['processed_text'] = parallelize_on_rows(df_train.clear_text, preprocess_text) 

In [ ]:
df_train.to_parquet("df_tarin.pq")